In [1]:
import os

# 下载解压数据集
#os.system("modelscope download --dataset ddzhu123/seq-monkey mobvoi_seq_monkey_general_open_corpus.jsonl.tar.bz2 --local_dir ./data")

# 解压预训练数据集
#os.system("tar -xjf ./data/mobvoi_seq_monkey_general_open_corpus.jsonl.tar.bz2 -C ./data/")

# 下载 sft 微调数据集
os.system(f'huggingface-cli download --repo-type dataset --resume-download BelleGroup/train_3.5M_CN --local-dir ./data/train_3.5M_CN')

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.


/root/autodl-tmp/conda/envs/HiChunk_w/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]Downloading 'train_3.5M_CN.json' to 'data/train_3.5M_CN/.cache/huggingface/download/XHAftARAQ0O1EwZpQc0vOYFR4hs=.cb8641f18fcf25bd22d33dac6531c73e899767140712893c6157e70936867089.incomplete' (resume from 73400320/4852568524)


/root/autodl-tmp/wzh/data/train_3.5M_CN


Download complete. Moving file to data/train_3.5M_CN/train_3.5M_CN.json
Fetching 3 files: 100%|██████████| 3/3 [09:14<00:00, 184.75s/it]


0

In [1]:
import json
from tqdm import tqdm
# 处理训练数据
def split_text(text, chunk_size=512):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

input_file = './data/mobvoi_seq_monkey_general_open_corpus.jsonl'

with open('seq_monkey_datawhale.jsonl', 'w', encoding='utf-8') as pretrain:
    with open(input_file,"r",encoding='utf-8') as f:
        data = f.readlines()
        for line in tqdm(data,desc="Processing data",leave=False):
            line = line.strip()
            if not line:
                continue
            try:
                item = json.loads(line)
                if 'text' not in item:
                    continue
                text = item['text']
                chunks = split_text(text, chunk_size=512)
                for chunk in chunks:
                    json_line = json.dumps({"text": chunk}, ensure_ascii=False)
                    pretrain.write(json_line + '\n')
            except json.JSONDecodeError:
                continue


# 2 处理SFT数据

# def convert_message(data):
#     """
#     将原始数据转换为标准格式
#     """
#     message = [
#         {"role": "system", "content": "你是一个AI助手"},
#     ]
#     for item in data:
#         if item['from'] == 'human':
#             message.append({'role': 'user', 'content': item['value']})
#         elif item['from'] == 'assistant':
#             message.append({'role': 'assistant', 'content': item['value']})
#     return message

# input_sft_file = './data/train_3.5M_CN/train_3.5M_CN.json'
# with open("BellGroup_sft.jsonl","a",encoding = "utf-8") as sft:
#     with open(input_sft_file,"r",encoding="utf-8") as f:
#         data = f.readlines()
#         for line in tqdm(data,desc="Processing SFT data",leave=False):
#             line = line.strip()
#             if not line:
#                 continue
#             try:
#                 item = json.loads(line)
#                 if 'conversations' not in item:
#                     continue
#                 messages = convert_message(item['conversations'])
#                 json_line = json.dumps({"messages": messages}, ensure_ascii=False)
#                 sft.write(json_line + '\n')
#             except json.JSONDecodeError:
#                 continue

In [2]:
import os
import json
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./custom_tokenizer")
with open("./BellGroup_sft.jsonl", "r") as file:
    data = file.readlines()
    for line in data[:5]:
        json_line = json.loads(line)
        text = tokenizer.apply_chat_template(json_line['messages'])
        
        print(json_line)
        print(tokenizer.decode(text))


{'messages': [{'role': 'system', 'content': '你是一个AI助手'}, {'role': 'user', 'content': '针对健身房的新手，设计一套适合他们的健身器械使用指南，包括安全应用、正确姿势等方面。'}, {'role': 'assistant', 'content': '健身器械使用指南\n1. 开始前，请先进行热身运动。这会帮助你的身体适应运动，并减少受伤的风险。\n2. 在使用健身器械前，确保你已经了解了其使用方法。请阅读说明书或咨询教练以获得正确的使用技巧。\n3. 谨防过度使用或过度挑战你的身体。 如果你觉得有些动作太难或太重，请添加锻炼计划，以逐步提高动作难度。\n4. 使用合适的装备。 确保你拥有合适的运动鞋和舒适的运动服。 不要在裸露的脚或短裤上进行重量训练。\n5. 在健身器械上使用安全装置。 这些通常用于保护你的身体免受不当操作造成的损伤。 例如，重量训练中，你需要使用杠铃和负重时，一定要使用卡子来防止重量滑落。\n6. 注意正确的姿势。 如果你的姿势是错误的，那么你的身体很容易被伤害到，你也可能无法获得最佳的锻炼效果。 至关重要的是，保持直立的身体，保持头部和颈部的稳定，并使用合适的重量。\n7. 保持合理的呼吸方式。 无论何时进行训练，都必须保持正常呼吸。 当你需要用力时，呼气； 当你放松时，吸气。\n8. 安全存放器械。 在使用健身器械后，你需要把它们归还给适当的位置，以便其他人可以使用它们。\n总之，健身器械的正确使用是关键之一，如果不健康和不安全，它们将无法帮助您达到您所需的健康成果。 选择适当的训练计划，并为训练提供足够的时间，以备逐渐适应新方法。 对于任何问题，请向教练咨询。'}]}
<|im_start|>system
你是一个AI助手<|im_end|>
<|im_start|>user
针对健身房的新手,设计一套适合他们的健身器械使用指南,包括安全应用、正确姿势等方面。<|im_end|>
<|im_start|>assistant
健身器械使用指南
1. 开始前,请先进行热身运动。这会帮助你的身体适应运动,并减少受伤的风险。
2. 在使用健身器械前,确保你已经了解了其使用方法。请阅读说明书或咨询教练以获得正确的使用技巧。
3. 谨防过度使用或过度挑战你的身体。 如果你